In [1]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, show, output_file, save, push_notebook, curdoc
from bokeh.plotting import figure, curdoc, gridplot
from bokeh.models import (DateRangeSlider, ColumnDataSource, HoverTool, CustomJS, 
                          Slider, Range1d, FactorRange, Legend, Label, 
                          LabelSet, ColorBar, NumeralTickFormatter, 
                          DatetimeTickFormatter, Toggle, CheckboxGroup, 
                          RadioButtonGroup, TextInput, Button, Div, Tabs)
from bokeh.layouts import row, column, gridplot, layout
from bokeh.palettes import HighContrast3, Viridis256, Category20, Inferno256, Cividis256, Turbo256
from bokeh.transform import factor_cmap, dodge, linear_cmap, log_cmap
from bokeh.events import DoubleTap, MouseMove, PanStart, Tap
# from bokeh.tile_providers import get_provider, Vendors
from bokeh.embed import components, file_html
from bokeh.resources import CDN
from bokeh.themes import Theme
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from bokeh.palettes import Category10
from bokeh.models import ColumnDataSource, HoverTool, RangeTool, Range1d, BoxAnnotation
from bokeh.io import output_file, curdoc, show
from bokeh.models import ColumnDataSource, HoverTool, Button, CustomJS, Div, RangeTool, BoxAnnotation
from bokeh.plotting import figure
from bokeh.layouts import column, row, gridplot
from bokeh.palettes import Category10, Category20
from bokeh.models import Panel, Tabs
import holoviews as hv
import numpy as np
import pandas as pd
from bokeh.models import Button, Div, CustomJS, ColumnDataSource, HoverTool, RangeTool, BoxAnnotation, Spacer


In [2]:
age_opts = ['6m', '9m', '12m', '18m']
loc_opts = ['STL', 'UNC', 'PHI']
clusters = [1, 2, 3]

x = np.linspace(1, 1000, 1000)
y = np.random.normal(5, 1, len(x))
c = np.random.choice(clusters, size = len(x))
age = np.random.choice(age_opts, size = len(x))
loc = np.random.choice(loc_opts, size = len(x))

data = pd.DataFrame({
    'x': x,
    'y': y,
    'c': c,
    'age' : age, 
    'loc' : loc
}, index=pd.Index(range(1, len(x) + 1), name='frame'))

In [3]:
output_notebook()
def prepare_wedge_data(column_name, colors):
    counts = data[column_name].value_counts()
    categories = sorted(counts.index)
    sizes = counts.loc[categories].values

    angles = np.linspace(0, 2 * np.pi, len(categories) + 1)
    start_angles = angles[:-1]
    end_angles = angles[1:]

    return pd.DataFrame({
        'start_angle': start_angles,
        'end_angle': end_angles,
        'color': colors,
        column_name: categories,
        'value': sizes
    })

source = ColumnDataSource(data)  # Reset index to include it as a column

# Define colors based on clusters
colors = Category10[3]  # Use a color palette for up to 10 clusters
data['color'] = data['c'].map(lambda cluster: colors[cluster-1])
source.add(data['color'], 'color')

# Define the output HTML file
output_file("data_visualization_dashboard.html")

# Donut Plot for Clusters
cluster_colors = Category10[3]
cluster_wedge_data = prepare_wedge_data('c', cluster_colors)

cluster_plot = figure(height=350, width=350, title="Donut Plot of Clusters", tools='hover',
                      tooltips="@c: @value", x_range=(-1, 1), y_range=(-1, 1))
cluster_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
                   line_color="white", fill_color='color', legend_field='c', source=cluster_wedge_data)
cluster_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
cluster_plot.axis.axis_label = None
cluster_plot.axis.visible = False
cluster_plot.grid.grid_line_color = None

# Donut Plot for Age
age_colors = Category20[len(data['age'].unique())]
age_wedge_data = prepare_wedge_data('age', age_colors)

age_plot = figure(height=350, width=350, title="Donut Plot of Age Groups", tools='hover',
                  tooltips="@age: @value", x_range=(-1, 1), y_range=(-1, 1))
age_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
               line_color="white", fill_color='color', legend_field='age', source=age_wedge_data)
age_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
age_plot.axis.axis_label = None
age_plot.axis.visible = False
age_plot.grid.grid_line_color = None

# Create a ColumnDataSource
source = ColumnDataSource(data)

# Define tooltips for hover
TOOLTIPS = [
    ("Frame", "@frame"),
    ("X", "@x"),
    ("Y", "@y"),
    ("Cluster", "@c"),
    ("Age", "@age"),
    ("Loc", "@loc"),
]

# Create the detailed scatter plot
detailed_plot = figure(
    title="Detailed Scatter Plot",
    tools="pan,wheel_zoom,box_zoom,reset,hover",
    width=800,
    height=400,
    tooltips=TOOLTIPS,
    background_fill_color="#efefef"
)
detailed_plot.scatter(x='x', y='y', color='color', source=source, size=10)
detailed_plot.add_tools(HoverTool(tooltips=TOOLTIPS))

# Create the minimap
minimap = figure(
    width=detailed_plot.width,
    height=150,
    tools="",
    toolbar_location=None,
    background_fill_color=detailed_plot.background_fill_color,
    title="Drag the middle or edges of the selection box below, or double click to start a new box"
)
minimap.scatter(x='x', y='y', color='color', source=source, size=10)
minimap.x_range.range_padding = 0
minimap.ygrid.grid_line_color = None

# Create and add the RangeTool
range_tool = RangeTool(x_range=detailed_plot.x_range, y_range=detailed_plot.y_range)
range_tool.overlay.fill_color = "darkblue"
range_tool.overlay.fill_alpha = 0.3
minimap.add_tools(range_tool)

# Create and add the BoxAnnotation for dragging
box = BoxAnnotation(
    left=detailed_plot.x_range.start,
    right=detailed_plot.x_range.end,
    top=detailed_plot.y_range.end,
    bottom=detailed_plot.y_range.start,
    fill_color="lightgrey",
    fill_alpha=0.3,
    editable=True  # Allow the box to be resized
)
minimap.add_layout(box)

# Create a button to toggle the minimap
toggle_button = Button(label="Toggle Minimap", button_type="success")
toggle_button.js_on_click(CustomJS(args=dict(minimap=minimap), code="""
    minimap.visible = !minimap.visible;
"""))

# Create multiple image placeholders
image_placeholder_1 = Div(text='<img src="https://via.placeholder.com/150" alt="Image 1" width="150" height="150">', width=150, height=150)
image_placeholder_2 = Div(text='<img src="https://via.placeholder.com/150" alt="Image 2" width="150" height="150">', width=150, height=150)
image_placeholder_3 = Div(text='<img src="https://via.placeholder.com/150" alt="Image 3" width="150" height="150">', width=150, height=150)

# Add a blurb of text with HTML and LaTeX
text_blurb = Div(text="""
    <h2>Data Visualization Dashboard</h2>
    <p>This dashboard includes various visualizations to help understand the dataset:</p>
    <ul>
        <li>Donut plots representing <b>clusters</b> and <b>age groups</b>.</li>
        <li>A detailed scatter plot with an interactive minimap.</li>
        <li>Multiple image placeholders for relevant graphics.</li>
        <li>Additional graph placeholders for further analysis.</li>
        <li>A video placeholder for visual content.</li>
    </ul>
    <p>Mathematical formula example: \( E = mc^2 \)</p>
""", width=800)

# Additional graph placeholders
additional_plot_1 = figure(height=350, width=350, title="Additional Graph 1")
additional_plot_2 = figure(height=350, width=350, title="Additional Graph 2")

# Arrange the donut plots side by side
donut_plots = row(cluster_plot, age_plot)

# Combine all components into a structured layout
layout = column(
    text_blurb,
    row(image_placeholder_1, image_placeholder_2, image_placeholder_3),
    donut_plots,
    row(additional_plot_1, additional_plot_2),
    detailed_plot,
    minimap,
    toggle_button,
    Div(text='<video width="320" height="240" controls><source src="your_video.mp4" type="video/mp4">Your browser does not support the video tag.</video>', width=320, height=240),
    sizing_mode='stretch_both'  # Allow responsive sizing
)

# Add layout to the document
curdoc().add_root(layout)

# Show the results
show(layout)


<>:123: SyntaxWarning: invalid escape sequence '\('
<>:123: SyntaxWarning: invalid escape sequence '\('
/var/folders/pf/49k8jg7948ddty_lg9b4pybr0000gn/T/ipykernel_1408/623267767.py:123: SyntaxWarning: invalid escape sequence '\('
  text_blurb = Div(text="""


Loading BokehJS ...

In [4]:
from bokeh.layouts import gridplot, column, row

# Prepare gridplot for arranging figures
grid = gridplot([[cluster_plot, age_plot],
                 [detailed_plot, minimap],
                 [additional_plot_1, additional_plot_2]])

# Add a video placeholder separately
video_placeholder = Div(text='<video width="320" height="240" controls>'
                             '<source src="your_video.mp4" type="video/mp4">'
                             'Your browser does not support the video tag.</video>', 
                        width=320, height=240)

# Combine grid layout and other components into a final layout
layout = column(
    text_blurb,
    row(image_placeholder_1, image_placeholder_2, image_placeholder_3),
    grid,  # Add gridplot containing your figures
    video_placeholder,  # Add video placeholder separately
    toggle_button,
    sizing_mode='stretch_both'  # Allow responsive sizing
)

# Add layout to the document
curdoc().add_root(layout)

# Show the results
show(layout)


In [5]:
def prepare_wedge_data(column_name, colors):
    counts = data[column_name].value_counts()
    categories = sorted(counts.index)
    sizes = counts.loc[categories].values

    angles = np.linspace(0, 2 * np.pi, len(categories) + 1)
    start_angles = angles[:-1]
    end_angles = angles[1:]

    return pd.DataFrame({
        'start_angle': start_angles,
        'end_angle': end_angles,
        'color': colors,
        column_name: categories,
        'value': sizes
    })

# Donut Plot for Clusters
cluster_colors = Category10[3]
cluster_wedge_data = prepare_wedge_data('c', cluster_colors)

cluster_plot = figure(height=350, width=350, title="Donut Plot of Clusters", tools='hover',
                      tooltips="@c: @value", x_range=(-1, 1), y_range=(-1, 1))
cluster_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
                   line_color="white", fill_color='color', legend_field='c', source=cluster_wedge_data)
cluster_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
cluster_plot.axis.visible = False
cluster_plot.grid.grid_line_color = None

# Donut Plot for Age
age_colors = Category20[len(data['age'].unique())]
age_wedge_data = prepare_wedge_data('age', age_colors)

age_plot = figure(height=350, width=350, title="Donut Plot of Age Groups", tools='hover',
                  tooltips="@age: @value", x_range=(-1, 1), y_range=(-1, 1))
age_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
               line_color="white", fill_color='color', legend_field='age', source=age_wedge_data)
age_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
age_plot.axis.visible = False
age_plot.grid.grid_line_color = None

# Scatter plot
source = ColumnDataSource(data)
detailed_plot = figure(
    title="Detailed Scatter Plot",
    tools="pan,wheel_zoom,box_zoom,reset,hover",
    width=800,
    height=400,
    background_fill_color="#efefef"
)
detailed_plot.scatter(x='x', y='y', color='color', source=source, size=10)

# Minimap
minimap = figure(
    width=detailed_plot.width,
    height=150,
    background_fill_color=detailed_plot.background_fill_color,
    title="Minimap"
)
minimap.scatter(x='x', y='y', color='color', source=source, size=10)
range_tool = RangeTool(x_range=detailed_plot.x_range, y_range=detailed_plot.y_range)
range_tool.overlay.fill_color = "darkblue"
range_tool.overlay.fill_alpha = 0.3
minimap.add_tools(range_tool)

# Additional graph placeholders
additional_plot_1 = figure(height=350, width=350, title="Additional Graph 1")
additional_plot_2 = figure(height=350, width=350, title="Additional Graph 2")

# Video placeholder
video_placeholder = Div(text='<video width="320" height="240" controls>'
                             '<source src="your_video.mp4" type="video/mp4">'
                             'Your browser does not support the video tag.</video>', 
                        width=320, height=240)

# Image placeholders
image_placeholder_1 = Div(text='<img src="https://via.placeholder.com/150" alt="Image 1" width="150" height="150">', width=150, height=150)
image_placeholder_2 = Div(text='<img src="https://via.placeholder.com/150" alt="Image 2" width="150" height="150">', width=150, height=150)
image_placeholder_3 = Div(text='<img src="https://via.placeholder.com/150" alt="Image 3" width="150" height="150">', width=150, height=150)

# Create toggle buttons for each object
def create_toggle_button(label, target):
    button = Button(label=f"Toggle {label}", button_type="success", width=150)
    button.js_on_click(CustomJS(args=dict(target=target), code="target.visible = !target.visible;"))
    return button

# Toggle buttons for each plot
toggle_buttons = [
    create_toggle_button("Cluster Plot", cluster_plot),
    create_toggle_button("Age Plot", age_plot),
    create_toggle_button("Detailed Plot", detailed_plot),
    create_toggle_button("Minimap", minimap),
    create_toggle_button("Additional Plot 1", additional_plot_1),
    create_toggle_button("Additional Plot 2", additional_plot_2),
    create_toggle_button("Video", video_placeholder),
    create_toggle_button("Image 1", image_placeholder_1),
    create_toggle_button("Image 2", image_placeholder_2),
    create_toggle_button("Image 3", image_placeholder_3)
]

# Arrange the donut plots side by side
donut_plots = row(cluster_plot, age_plot)

# Combine all components into a structured layout
layout = column(
    row(image_placeholder_1, image_placeholder_2, image_placeholder_3),
    donut_plots,
    row(additional_plot_1, additional_plot_2),
    detailed_plot,
    minimap,
    video_placeholder,
    column(*toggle_buttons),  # Add the toggle buttons
    sizing_mode='stretch_both'  # Allow responsive sizing
)

# Add layout to the document
curdoc().add_root(layout)

# Show the results
show(layout)

In [6]:
def create_toggleable_object(label, obj, button_position="below"):
    """
    Encapsulates an object with a toggle button of the same width or height.
    
    Args:
        label (str): The label for the toggle button.
        obj (Bokeh model): The object to encapsulate (e.g., a plot, image, video, etc.).
        button_position (str): Whether to place the button 'below' or 'above' the object.
        
    Returns:
        layout (Bokeh layout): A layout with the object and a toggle button.
    """
    # Create the toggle button
    if button_position == "below":
        button_width = obj.width
        button_height = 50  # Set a fixed height for the button
        button = Button(label=f"Toggle {label}", button_type="success", width=button_width)
    elif button_position == "above":
        button_width = 150  # Set a fixed width for the button
        button_height = obj.height
        button = Button(label=f"Toggle {label}", button_type="success", height=button_height)

    # Define the callback to toggle visibility
    button.js_on_click(CustomJS(args=dict(target=obj), code="target.visible = !target.visible;"))

    # Arrange the object and button
    if button_position == "below":
        layout = column(obj, button)
    elif button_position == "above":
        layout = column(button, obj)
    elif button_position == "left":
        layout = row(button, obj)
    elif button_position == "right":
        layout = row(obj, button)

    return layout

In [7]:
# Define color palette
colors = Category10[3]
data['color'] = data['c'].map(lambda cluster: colors[cluster-1])
source = ColumnDataSource(data)

# Donut Plot for Clusters
cluster_colors = Category10[3]
cluster_wedge_data = prepare_wedge_data('c', cluster_colors)

cluster_plot = figure(height=350, width=350, title="Donut Plot of Clusters", tools='hover',
                      tooltips="@c: @value", x_range=(-1, 1), y_range=(-1, 1))
cluster_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
                   line_color="white", fill_color='color', legend_field='c', source=cluster_wedge_data)
cluster_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
cluster_plot.axis.visible = False
cluster_plot.grid.grid_line_color = None

# Donut Plot for Age
age_colors = Category20[len(data['age'].unique())]
age_wedge_data = prepare_wedge_data('age', age_colors)

age_plot = figure(height=350, width=350, title="Donut Plot of Age Groups", tools='hover',
                  tooltips="@age: @value", x_range=(-1, 1), y_range=(-1, 1))
age_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
               line_color="white", fill_color='color', legend_field='age', source=age_wedge_data)
age_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
age_plot.axis.visible = False
age_plot.grid.grid_line_color = None

# Detailed scatter plot
TOOLTIPS = [
    ("Frame", "@frame"),
    ("X", "@x"),
    ("Y", "@y"),
    ("Cluster", "@c"),
    ("Age", "@age"),
    ("Loc", "@loc"),
]

detailed_plot = figure(
    title="Detailed Scatter Plot",
    tools="pan,wheel_zoom,box_zoom,reset,hover",
    width=800,
    height=400,
    tooltips=TOOLTIPS,
    background_fill_color="#efefef"
)
detailed_plot.scatter(x='x', y='y', color='color', source=source, size=10)

# Minimap
minimap = figure(
    width=detailed_plot.width,
    height=150,
    tools="",
    toolbar_location=None,
    background_fill_color=detailed_plot.background_fill_color,
    title="Minimap"
)
minimap.scatter(x='x', y='y', color='color', source=source, size=10)

range_tool = RangeTool(x_range=detailed_plot.x_range, y_range=detailed_plot.y_range)
range_tool.overlay.fill_color = "darkblue"
range_tool.overlay.fill_alpha = 0.3
minimap.add_tools(range_tool)

box = BoxAnnotation(
    left=detailed_plot.x_range.start,
    right=detailed_plot.x_range.end,
    top=detailed_plot.y_range.end,
    bottom=detailed_plot.y_range.start,
    fill_color="lightgrey",
    fill_alpha=0.3,
    editable=True
)
minimap.add_layout(box)

# Text and Image placeholders
text_blurb = Div(text="<h2>Data Visualization Dashboard</h2>", width=800)

image_placeholder_1 = Div(text='<img src="https://via.placeholder.com/150" alt="Image 1" width="150" height="150">')
image_placeholder_2 = Div(text='<img src="https://via.placeholder.com/150" alt="Image 2" width="150" height="150">')
image_placeholder_3 = Div(text='<img src="https://via.placeholder.com/150" alt="Image 3" width="150" height="150">')

# Encapsulate all objects with toggle buttons
toggleable_cluster_plot = create_toggleable_object("Cluster Donut Plot", cluster_plot, button_position="below")
toggleable_age_plot = create_toggleable_object("Age Donut Plot", age_plot, button_position="below")
toggleable_detailed_plot = create_toggleable_object("Detailed Scatter Plot", detailed_plot, button_position="below")
toggleable_minimap = create_toggleable_object("Minimap", minimap, button_position="below")
toggleable_text = create_toggleable_object("Dashboard Text", text_blurb, button_position="below")
toggleable_image_1 = create_toggleable_object("Image 1", image_placeholder_1, button_position="below")
toggleable_image_2 = create_toggleable_object("Image 2", image_placeholder_2, button_position="below")
toggleable_image_3 = create_toggleable_object("Image 3", image_placeholder_3, button_position="below")

# Arrange everything in a layout
layout = column(
    toggleable_text,
    row(toggleable_image_1, toggleable_image_2, toggleable_image_3),
    row(toggleable_cluster_plot, toggleable_age_plot),
    toggleable_detailed_plot,
    toggleable_minimap
)

# Add layout to document
curdoc().add_root(layout)

# Show the results (if running locally)
show(layout)  # Uncomment this if running in a local environment

In [11]:
# Set output file
output_file("data_visualization_dashboard_with_toggle.html")

# Define color palette
colors = Category10[3]
data['color'] = data['c'].map(lambda cluster: colors[cluster-1])
source = ColumnDataSource(data)

# Donut Plot for Clusters
cluster_colors = Category10[3]
cluster_wedge_data = prepare_wedge_data('c', cluster_colors)

cluster_plot = figure(height=350, width=350, title="Donut Plot of Clusters", tools='hover',
                      tooltips="@c: @value", x_range=(-1, 1), y_range=(-1, 1))
cluster_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
                   line_color="white", fill_color='color', legend_field='c', source=cluster_wedge_data)
cluster_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
cluster_plot.axis.visible = False
cluster_plot.grid.grid_line_color = None

# Donut Plot for Age
age_colors = Category20[len(data['age'].unique())]
age_wedge_data = prepare_wedge_data('age', age_colors)

age_plot = figure(height=350, width=350, title="Donut Plot of Age Groups", tools='hover',
                  tooltips="@age: @value", x_range=(-1, 1), y_range=(-1, 1))
age_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
               line_color="white", fill_color='color', legend_field='age', source=age_wedge_data)
age_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
age_plot.axis.visible = False
age_plot.grid.grid_line_color = None

# Detailed scatter plot
TOOLTIPS = [
    ("Frame", "@frame"),
    ("X", "@x"),
    ("Y", "@y"),
    ("Cluster", "@c"),
    ("Age", "@age"),
    ("Loc", "@loc"),
]

detailed_plot = figure(
    title="Detailed Scatter Plot",
    tools="pan,wheel_zoom,box_zoom,reset,hover",
    width=800,
    height=400,
    tooltips=TOOLTIPS,
    background_fill_color="#efefef"
)
detailed_plot.scatter(x='x', y='y', color='color', source=source, size=10)

# Minimap
minimap = figure(
    width=detailed_plot.width,
    height=150,
    tools="",
    toolbar_location=None,
    background_fill_color=detailed_plot.background_fill_color,
    title="Minimap"
)
minimap.scatter(x='x', y='y', color='color', source=source, size=10)

range_tool = RangeTool(x_range=detailed_plot.x_range, y_range=detailed_plot.y_range)
range_tool.overlay.fill_color = "darkblue"
range_tool.overlay.fill_alpha = 0.3
minimap.add_tools(range_tool)

box = BoxAnnotation(
    left=detailed_plot.x_range.start,
    right=detailed_plot.x_range.end,
    top=detailed_plot.y_range.end,
    bottom=detailed_plot.y_range.start,
    fill_color="lightgrey",
    fill_alpha=0.3,
    editable=True
)
minimap.add_layout(box)

# Text and Image placeholders
text_blurb = Div(text="<h2>Data Visualization Dashboard</h2>", width=800)

image_placeholder_1 = Div(text='<img src="https://via.placeholder.com/150" alt="Image 1" width="150" height="150">')
image_placeholder_2 = Div(text='<img src="https://via.placeholder.com/150" alt="Image 2" width="150" height="150">')
image_placeholder_3 = Div(text='<img src="https://via.placeholder.com/150" alt="Image 3" width="150" height="150">')

# Encapsulate all objects with toggle buttons
toggleable_cluster_plot = create_toggleable_object("Cluster Donut Plot", cluster_plot, button_position="below")
toggleable_age_plot = create_toggleable_object("Age Donut Plot", age_plot, button_position="below")
toggleable_detailed_plot = create_toggleable_object("Detailed Scatter Plot", detailed_plot, button_position="below")
toggleable_minimap = create_toggleable_object("Minimap", minimap, button_position="below")
toggleable_text = create_toggleable_object("Dashboard Text", text_blurb, button_position="below")
toggleable_image_1 = create_toggleable_object("Image 1", image_placeholder_1, button_position="below")
toggleable_image_2 = create_toggleable_object("Image 2", image_placeholder_2, button_position="below")
toggleable_image_3 = create_toggleable_object("Image 3", image_placeholder_3, button_position="below")

# Centering and neat layout
spacer = Spacer(width=100)

# Create symmetric and neat layout
layout = column(
    toggleable_text,
    row(spacer, toggleable_image_1, spacer, toggleable_image_2, spacer, toggleable_image_3, spacer, sizing_mode="stretch_both"),
    row(spacer, toggleable_cluster_plot, spacer, toggleable_age_plot, spacer, sizing_mode="stretch_both"),
    toggleable_detailed_plot,
    toggleable_minimap,
    sizing_mode="scale_both"
)

# Add layout to document
curdoc().add_root(layout)

# Show the results (if running locally)
show(layout)  # Uncomment this if running in a local environment

ERROR:bokeh.core.validation.check:E-1027 (REPEATED_LAYOUT_CHILD): The same model can't be used multiple times in a layout: Row(id='p2895', ...)
ERROR:bokeh.core.validation.check:E-1027 (REPEATED_LAYOUT_CHILD): The same model can't be used multiple times in a layout: Row(id='p2896', ...)


ERROR:bokeh.core.validation.check:E-1027 (REPEATED_LAYOUT_CHILD): The same model can't be used multiple times in a layout: Row(id='p2895', ...)
ERROR:bokeh.core.validation.check:E-1027 (REPEATED_LAYOUT_CHILD): The same model can't be used multiple times in a layout: Row(id='p2896', ...)
